In [14]:
import sympy as sym 
from IPython.display import display, Math
from itertools import permutations

In [15]:
"""

Results down below.

Workflow: 
    1.) Set 'disturbed_body_hn' and 'maximal_hn'
    2.) 'find_all_disturbers'
    3.) =>  'left_hand_sides' == 'right_hand_sides' and solve
    4.) 'traverse_tree_and_replace_indices'
"""
ALL = ['latex_this', 'find_all_disturbers', 'left_hand_sides', 'right_hand_side', 'traverse_tree_and_replace_indices']

In [16]:
# -------- F U N C T I O N S    R E P R E S E N T I N G    C O N S T A N T S -------- #
def kappa(i: int, j:int) -> sym.core.symbol.Symbol:
    """
    Function representing the κ_{ij} constant. 
    
    :param i:         The hierarchy number of the tidal disturber.
    :param j:         The hierarchy number of the disturbed body.      
    
    :return: expr         
    """
    g = sym.symbols("G")
    m_alpha = sym.symbols('m_{' + str(i) + '}')
    k_beta = sym.symbols('k_{L' + str(j)+ '}')
    r_beta = sym.symbols('R_{' + str(j)+ '}')
    
    return g * m_alpha**2 * k_beta * r_beta**5

def dirac(alpha: int, beta: int) -> int:
    """
    Represents a kronecker-delta. Returns zero if alpha is not equal to beta and one otherwise.
    
    :param alpha:         
    :param beta:               
    
    :return:          
    """
    num = None
    if alpha == beta:
        num = 1
    elif alpha != beta:
        num = 0
    else: 
        raise Warning("Something went wrong, inside 'dirac' function call.")
    return num
    
def lamda(i: int, j:int, k:int, l:int) -> tuple:
    """
    Function representing the λ_{ijk} constant. Returns a tuple, where the first element represents the constant for the spin frequency derivative
    and the second element for the orbit frequency derivative.
    
    :param i:         The hierarchy number of the tidal disturber.
    :param j:         The hierarchy number of the disturbed body.
    :param k:         The hierarchy number of that tidal disturber, the derivative is currently taken with respect to 
                        on the left hand side of the ELE.      
    :param l:         The hierarchy number of that tidally disturbed body, the derivative is currently taken with respect 
                        to on the left hand side of the ELE.
    
    :return: expr         
    """
    signum = sym.Symbol(r'\mathrm{sgn}('+ r'\Omega_{' +str(j) + '}-n_{' + str(i) + '-' + str(j) + '})')
    quality_factor = sym.Symbol(f'Q_{j}')
    const_spin = dirac(alpha=l, beta=j)
    const_orbit = - dirac(alpha=l, beta=j) * dirac(alpha=k, beta=i) 
    base_expr = sym.Rational(-3/2) * kappa(i=i, j=j) * 1/quality_factor * signum
    lamda_spin =  base_expr * const_spin
    lamda_orbit = base_expr * const_orbit
    
    return lamda_spin, lamda_orbit


In [17]:
# -------- M A T H E M A T I C A L    M A N I P U L A T E R S -------- #

def find_all_disturbers(disturbed_body: int, size_of_system: int) -> list:
    """
    Based on the hierarchy number of the disturbed body, this returns a list of all hierarchy numbers associated with the disturbers of that body.
    These are basically just the "left" and "right" neighbours of a given body. Returns e.g. [1,3] for inputs (2,3) or [None, 2] for (1,3). 
    
    :param disturbed_body:          The hierarchy number of the disturbed body.
    :param size_of_system:          The biggest hierarchy number of the system, characterizes its size.      
    
    :return: all_disturbers         A list of all hierarchy numbers that correspond to bodies disturbing 'disturbed_body'.
    """
    all_disturbers = []
    left_disturber = None
    right_disturber = None
    left_boundary = 1
    right_boundary = size_of_system
    
    left_neighbour = disturbed_body - 1
    if left_neighbour >= left_boundary:
        left_disturber = left_neighbour
    
    right_neighbour = disturbed_body + 1
    if right_neighbour <= right_boundary:
        right_disturber = right_neighbour
    
    if left_disturber is not None:
        all_disturbers.append(left_disturber)
    if right_disturber is not None:
        all_disturbers.append(right_disturber)
    
    return all_disturbers

def left_hand_sides(disturbed_body: int, disturber:int , print_debug: bool) -> tuple:
    """
    Returns a tuple containing sympy expressions representing the left hand sides of the Euler-Lagrange-Equations.
    The first element of the tuple is the LHS for the spin-frequency-equation and the second for the orbit-frequency-equation.  
    
    :param disturbed_body:          The hierarchy number of the disturbed body.
    :param disturber:               The hierarchy number of the disturber body. Determines constants on the lhs of the orbit-frequency-equation.   
    :param print_debug:             Prints the latex representation of the left and right hand sides. 
    
    :return: lhs_spin, lhs_orbit    The LHS of the ELE packaged as a tuple for spin and orbit coordinates.
    """
    
    t = sym.symbols('t')
    omega_j = sym.Function(r'\Omega_{' + f'{disturbed_body}' + '}')(t)
    inertia_j = sym.symbols('I_{' + f'{disturbed_body}'+ '}')
    
    lhs_spin = inertia_j * omega_j.diff(t)
    
    disturber_mass = sym.symbols('m_{' + f'{disturber}'+ '}')
    disturbed_mass = sym.symbols('m_{' + f'{disturbed_body}'+ '}')
    mu = sym.symbols(r'\mu_{' + f'{disturber}' + '-' + f'{disturbed_body}' + '}')
    sm_axis = sym.Function('a_{' + f'{disturber}' + '-' + f'{disturbed_body}' + '}')(t)
    
    lhs_orbit = sym.Rational(1 / 2) * disturber_mass * disturbed_mass / (disturber_mass + disturbed_mass) * mu**(sym.Rational(1,2)) * sm_axis**(sym.Rational(-1,2))*sm_axis.diff(t)
    
    if print_debug:
        print("Debug statement of the function 'left_hand_sides'. Parameters:")
        print({'disturbed_body': disturbed_body, 'disturber': disturber, 'print_debug': print_debug})
        print("View the documentation for a description of the parameters.")
        print("Left hand side of ELE, spin equation:")
        latex_this(lhs_spin)
        print("Left hand side of ELE, orbit equation:")
        latex_this(lhs_orbit)
    
    return lhs_spin, lhs_orbit

def right_hand_side(disturbed_body: int, actual_disturber:int, all_disturbers: list , print_debug: bool) -> tuple:
    """
    Returns a tuple containing sympy expressions representing the right hand side of the Euler-Lagrange-Equations for the spin frequency 
    and orbit frequency coordinate, respectively the first and second element of the tuple..
    
    :param disturbed_body:          The hierarchy number of the disturbed body.
    :param actual_disturber:        The hierarchy number of the actual disturber. Determines which summand to collapse in the lamda constant.   
                                    Has to be the same input as 'disturber' argument in the function 'left_hand_sides'.
    :param all_disturbers:          A list containing the hierarchy number of all disturbers of 'disturbed_body'.    
    :param print_debug:             Prints the latex representation of the right hand side.
    
    :return: rhs                    A tuple containing the right hand sides of the ELE for spin and orbit coordinate.
    """
    
    t = sym.symbols("t")
    # this represents the sum over all {i}.
    helper_list_spin = []
    helper_list_orbit = []
    for possible_disturber in all_disturbers:
        sm_axis = sym.Function('a_{' + f'{possible_disturber}' + '-' + f'{disturbed_body}' + '}')(t)
        constant_1_spin, constant_1_orbit = lamda(i=possible_disturber,j=disturbed_body,k=actual_disturber, l=disturbed_body)
        expr_spin = constant_1_spin * 1 / sm_axis ** 6 
        expr_orbit = constant_1_orbit * 1 / sm_axis ** 6 
        helper_list_spin.append(expr_spin)
        helper_list_orbit.append(expr_orbit)
        

    
    rhs_spin = sym.Add(*helper_list_spin)
    rhs_orbit = sym.Add(*helper_list_orbit)
    
    # Finally replace 'G', the gravitational constant in the orbit frequency case with mu_{i-j} / (m_i + m_j)
    mu = sym.symbols(r'\mu_{' + f'{actual_disturber}' + '-' + f'{disturbed_body}' + '}')
    disturber_mass = sym.symbols('m_{' + f'{actual_disturber}'+ '}')
    disturbed_mass = sym.symbols('m_{' + f'{disturbed_body}'+ '}')
    # find the instance of 'G in 'rhs_orbit'
    instance_of_g = sym.symbols("G")
    rhs_orbit = rhs_orbit.subs(instance_of_g, mu / (disturber_mass+disturbed_mass))
    
    if print_debug:
        print("Debug statement of the function 'left_hand_sides'. Parameters:")
        print({'disturbed_body': disturbed_body, 'actual_disturber': actual_disturber, 'all_disturbers':all_disturbers,'print_debug': print_debug})
        print("View the documentation for a description of the parameters.")
        print("Right hand side of ELE Spin Frequency:")
        latex_this(rhs_spin)
        print("Right hand side of ELE Orbit Frequency:")
        latex_this(rhs_orbit)
        

    
    return rhs_spin, rhs_orbit

def solve(lhs_spin: sym.core.symbol.Symbol, lhs_orbit: sym.core.symbol.Symbol, 
          rhs_spin: sym.core.symbol.Symbol, rhs_orbit: sym.core.symbol.Symbol,
          disturbed_body: int, current_disturber: int) -> tuple:
    """
    Takes instances of left and right hand sides of the ELE for different coordinates, sets equal and solves for quantities of interest.
    The tuple returns the solution to the spin frequency equation as its first element and the solution to the orbit frequency equation as its second.
    
    :param lhs_spin 
    :param lhs_orbit 
    :param rhs_spin 
    :param rhs_orbit 
    :param disturbed_body:      The hierarchy number of the disturbed body.
    :param current_disturber:   The hierarchy number of the current disturber ('k').
    
    :return: None
    """
    
    
    t = sym.symbols('t')
    omega_j = sym.Function(r'\Omega_{' + f'{disturbed_body}' + '}')(t)
    sm_axis = sym.Function('a_{' + f'{current_disturber}' + '-' + f'{disturbed_body}' + '}')(t)
    
    spin_equ = sym.Eq(lhs_spin, rhs_spin)
    solution_spin = sym.solve(spin_equ, omega_j.diff(t))
    
    orbit_equ = sym.Eq(lhs_orbit, rhs_orbit)
    solution_orbit = sym.solve(orbit_equ, sm_axis.diff(t))
    
    
    return solution_spin, solution_orbit

def find_all_equations(disturbed_body :int, disturber: int, all_disturbers: list, print_debug: bool) -> tuple:
    """
    Uses the 'solve' function to find all differential equations of interest.
    
    :param disturbed_body:      The hierarchy number of the disturbed body.
    :param disturber:           The hierarchy number of the current disturber. Used only to fix the indices of the semi-major-axis. 
    :param all_disturbers:      A list containing the hierarchy number of all disturbers of 'disturbed_body'. 
    :param print_debug:         Print debug statements.       
    
    :return: None
    """
    lhs_spin_instance, lhs_orbit_instance, rhs_spin_instance, rhs_orbit_instance = [None] * 4 
    k = disturber
    lhs_spin_instance, lhs_orbit_instance = left_hand_sides(disturbed_body = disturbed_body, disturber=k, print_debug=False)
    rhs_spin_instance, rhs_orbit_instance = right_hand_side(disturbed_body =disturbed_body, actual_disturber=k, 
                                                                all_disturbers=all_disturbers, print_debug=False)
    
    spin_solution, orbit_solution = solve(lhs_spin=lhs_spin_instance, lhs_orbit=lhs_orbit_instance, rhs_spin=rhs_spin_instance,
                                          rhs_orbit=rhs_orbit_instance, disturbed_body=disturbed_body, current_disturber=disturber)
    
    t=sym.symbols("t")
    semi_major_axis = sym.Function('a_{' + f'{disturber}' + '-' + f'{disturbed_body}' + '}')(t)
    omega_j = sym.Function(r'\Omega_{' + f'{disturbed_body}' + '}')(t)
   
    if print_debug:
        
        print("Debug statement of the function 'find_all_equations'. Parameters:")
        print({'disturbed_body': disturbed_body, 'disturber': disturber, 'all_disturbers':all_disturbers,'print_debug': print_debug})
        print("View the documentation for a description of the parameters.")
        
        print("Orbit Solution: ")
        latex_this(orbit_solution)
        display(Math(sym.latex(semi_major_axis.diff(t))+"="+sym.latex(orbit_solution[0])))
    
        print("Spin Solution: ")
        display(Math(sym.latex(omega_j.diff(t))+"="+sym.latex(spin_solution[0])))
    
    return sym.Eq(semi_major_axis.diff(t), orbit_solution[0]), sym.Eq(omega_j.diff(t), spin_solution[0])
    


In [18]:

# -------- S T Y L I S T I C    M A N I P U L A T E R S -------- #
def traverse_tree_and_replace_indices(outer_expr: sym.core.symbol.Symbol) -> sym.core.symbol.Symbol:
    
    """
    Uses recursive tree traversing to find all numerical indices in a sympy expression and replace them with their corresponding 
    latex letter index abbreviation. Returns the new sympy expression.
    
    :param outer_expr:          The expression to recursively traverse.
    
    :return: sym.core.symbol.Symbol
    """
    
    def inner_traverse_time_dependent_derivative(expr, global_expr):
        replacement_indices = [r'\mathrm{s}',r'\mathrm{p}',r'\mathrm{m}',r'\mathrm{sm}',r'\mathrm{ssm}',r'\mathrm{sssm}']
        for node in expr.args:
                             
            if node.is_Derivative:
                base =  node.args[0].name
                variable_name, indices = base.split("_")
                indices = indices [1:-1] # remove the first { and last }
                if 'O' not in variable_name:
                    # We are replacing a_{i-j} => two replacements
                    string_index_1 = indices[0]
                    string_index_2 = indices[2] # in the middle ,there is the '-' character
                    translation_1 = replacement_indices[int(string_index_1)-1]
                    translation_2 = replacement_indices[int(string_index_2)-1]
                    replacement = sym.symbols(r'\dot{'+variable_name+'}_{' + translation_1 + '-' + translation_2 + '}')
                else:
                    # 'Ω' is in base: We are replacing only one index \Omega_j
                     string_index_1 = indices[0]
                     translation_1 = replacement_indices[int(string_index_1)-1]
                     replacement = sym.symbols(r'\dot{'+variable_name+'}_{' + translation_1  + '}')
                return global_expr.subs(node, replacement)
            else:
                inner_traverse_time_dependent_derivative(node, global_expr)
  
    def replace_time_dependent_powers(list_of_nodes, list_of_replacements, global_expr):
        modified_expr = None
        for node, replacement in zip(list_of_nodes, list_of_replacements): 
            if modified_expr is None:
                modified_expr = global_expr.subs(node, replacement)
            else:
                modified_expr = modified_expr.subs(node, replacement)
        return modified_expr
  
    def inner_traverse_time_dependent_power(expr, global_expr, found_nodes=None, found_replacements=None):
 
        if found_nodes is None:
            found_nodes = []
        if found_replacements is None:
            found_replacements = []
        
        replacement_indices = [r'\mathrm{s}',r'\mathrm{p}',r'\mathrm{m}',r'\mathrm{sm}',r'\mathrm{ssm}',r'\mathrm{sssm}']
        for node in expr.args:
            
            if node.is_Pow and node.base.is_Function:
                # some kind of power on the base of a time-dependent a_{i-j}. Extract the indices
                base, exponent = node.as_base_exp()
                base = base.name
                variable_name, indices = base.split("_")
                indices = indices [1:-1] # remove the first { and last }
                string_index_1 = indices[0]
                string_index_2 = indices[2] # in the middle ,there is the '-' character
                translation_1 = replacement_indices[int(string_index_1)-1]
                translation_2 = replacement_indices[int(string_index_2)-1]
                replacement = sym.symbols(variable_name+'_{' + translation_1 + '-' + translation_2 + '}^{'+ str(exponent) +'}')
                found_nodes.append(node)
                found_replacements.append(replacement)
                #modified_expr = global_expr.subs(node, replacement)
                #return modified_expr, False # Return the modified expression and set the replaced flag to True
                    
            else:
                inner_traverse_time_dependent_power(node, global_expr, 
                                                                    found_nodes, found_replacements)
                
        return found_nodes, found_replacements
  
    def inner_traverse(expr):
        replacement_indices = [r'\mathrm{s}',r'\mathrm{p}',r'\mathrm{m}',r'\mathrm{sm}',r'\mathrm{ssm}',r'\mathrm{sssm}']
        for node in expr.args:
            if node.is_Symbol and '^' not in node.name:
                # a_{sm-p}^{-6} is already handled by other functions. if 'and '^' not in node.name' not specified,
                # the code takes the number it finds (6) and therefore replaces the indices with 'sssm'.
                # Building block => Rename
                # Index found
                base_symbol, *numerical_indices  = node.name.split("_")
                numerical_indices = " ".join(numerical_indices)
                list_of_digits_in_numerical_indices = [char for char in numerical_indices if char.isdigit()]
                number_of_indices = len(list_of_digits_in_numerical_indices)
                
                if number_of_indices == 0:
                    pass
                elif number_of_indices == 1:
                    # There exists only one simple index => Rename
                    numerical_index = int(list_of_digits_in_numerical_indices[0])
                    node.name = base_symbol + "_" + replacement_indices[int(numerical_index)-1]
                elif number_of_indices == 2:
                    # There exists more than one index! The base_symbol_{i-j} convention was followed.
                    string_index_1 = list_of_digits_in_numerical_indices[0]
                    string_index_2 = list_of_digits_in_numerical_indices[1]
                    translation_1 = replacement_indices[int(string_index_1)-1]
                    translation_2 = replacement_indices[int(string_index_2)-1]
                    # => Rename
                    node.name = base_symbol + '_{' + translation_1 + '-' + translation_2 + '}'
                elif number_of_indices == 3 and '^' not in node.name:
                    # very special case for the signum function
                    string_index_1 = list_of_digits_in_numerical_indices[0]
                    string_index_2 = list_of_digits_in_numerical_indices[1]
                    string_index_3 = list_of_digits_in_numerical_indices[2]
                    translation_1 = replacement_indices[int(string_index_1)-1]
                    translation_2 = replacement_indices[int(string_index_2)-1]
                    translation_3 = replacement_indices[int(string_index_3)-1]
                    node.name = r'\mathrm{sgn}(' + r'\Omega_{' + translation_1 + '}-n_{' + translation_2 + '-' + translation_3 + '}' + ')'
                elif number_of_indices == 3 and '^'  in node.name:
                    # already handled by functions 'inner_traverse_time_dependent_power' and 'inner_traverse_time_dependent_derivative'
                    pass
                else:
                    raise Warning("Something went wrong inside 'traverse_tree_and_replace_indices' function call. More than one index "
                                  f"found in base symbol. Found {number_of_indices} indices, in expression {node}.")
                
        
            else:
                inner_traverse(node)
        return expr
    
    power_nodes, power_replacements = inner_traverse_time_dependent_power(outer_expr,outer_expr)

    power_modification = replace_time_dependent_powers(power_nodes, power_replacements, outer_expr)
    derivative_and_power_modification = inner_traverse_time_dependent_derivative(power_modification, 
                                                                                 power_modification)    
    return inner_traverse(derivative_and_power_modification)
            
            
def latex_this(expr: sym.core.symbol.Symbol) -> None:
    """
    Takes a sympy expression, extracts the latex representation and parses it through Math, such that IPython can display it.
    
    :param expr: The sympy expression to evaluate.
    
    :return: None
    """
    display(Math(r'\LARGE'+sym.latex(expr)))
    
    
    

In [19]:
"""
Workflow: 
    1.) Set 'disturbed_body_hn' and 'maximal_hn'
    2.) 'find_all_disturbers'
    3.) =>  'left_hand_sides' == 'right_hand_sides' and solve
    4.) 'traverse_tree_and_replace_indices'
"""

"""disturbed_body_hn = 2 # The hierarchy number of the disturbed body.
maximal_hn = 3 # The biggest hierarchy number, characterizes the size of the system

all_disturbers_hn = find_all_disturbers(disturbed_body = disturbed_body_hn, size_of_system = maximal_hn)

for i, curr_disturber in enumerate(all_disturbers_hn):
    print("i ", [curr_disturber, disturbed_body_hn][1])
    disturber, disturbed = [curr_disturber, disturbed_body_hn][i], [curr_disturber, disturbed_body_hn][-i+1] # actio reactio
    
    print("all disturbing bodies: ", all_disturbers_hn, "disturbed: ", disturbed, " calculating for disturber ", disturber)
    orbit_equation, spin_equation = find_all_equations(disturbed_body=disturbed, disturber=disturber, all_disturbers=all_disturbers_hn, print_debug=False)
    orbit_equation = traverse_tree_and_replace_indices(orbit_equation)
    spin_equation  = traverse_tree_and_replace_indices(spin_equation)
    
    latex_this(orbit_equation)
    latex_this(spin_equation)"""


maximal_hn = 4 # The biggest hierarchy number, characterizes the size of the system

whole_system_hn = list(range(1, maximal_hn+1))
permutations = list(permutations(whole_system_hn, 2)) # Generate all permutations of length 2
# Filter out tuples where the difference is greater than 1, i.e. not nearest neighbour => No effect.
permutations_filtered = [(a, b) for a, b in permutations if abs(a - b) <= 1]
print("The whole system is a hierarchy of: ", whole_system_hn)
for interaction_pair in permutations_filtered:
    
    disturbed_body_hn, curr_disturber = interaction_pair
    print("disturbed body: ", disturbed_body_hn, " disturber: ", curr_disturber)
    
    all_disturbers_hn = find_all_disturbers(disturbed_body = disturbed_body_hn, size_of_system = maximal_hn)
    orbit_equation, spin_equation = find_all_equations(disturbed_body=disturbed_body_hn, disturber=curr_disturber, all_disturbers=all_disturbers_hn, print_debug=False)
    orbit_equation = traverse_tree_and_replace_indices(orbit_equation)
    spin_equation  = traverse_tree_and_replace_indices(spin_equation)
    
    latex_this(orbit_equation)
    latex_this(spin_equation)


The whole system is a hierarchy of:  [1, 2, 3, 4]
disturbed body:  1  disturber:  2


<IPython.core.display.Math object>

<IPython.core.display.Math object>

disturbed body:  2  disturber:  1


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

disturbed body:  4  disturber:  3


<IPython.core.display.Math object>

<IPython.core.display.Math object>